In [4]:
import os
import subprocess
import pandas as pd

#----- Set working directory to the root of the git repository -----
current_dir = os.getcwd()
git_root = subprocess.check_output(["git", "rev-parse", "--show-toplevel"], cwd=current_dir)
git_root = git_root.decode("utf-8").strip()
os.chdir(git_root)
new_dir = os.getcwd()
fpath_data = os.path.join(new_dir, "data/output.parquet")

In [5]:
import utils.weather

## Data Cleaning

In [6]:
df = pd.read_parquet(fpath_data)

# Temperature
df = df[(df["tmp"] > -20) & (df["tmp"] < 50)]

# Humidity
df = df[(df["hum"] > 0) & (df["hum"] < 100)]

# CO2
df = df[(df["CO2"] > 0) & (df["CO2"] < 5000)]

# VOC
df = df[(df["VOC"] > 0) & (df["VOC"] < 1000)]

# Remove unneeded columns
df = df.drop(columns=[
    "device_id", 
    "vis", 
    "IR", 
    "WIFI", 
    "BLE", 
    "rssi", 
    "channel_rssi", 
    "snr", 
    "gateway", 
    "channel_index", 
    "spreading_factor", 
    "bandwidth", 
    "f_cnt"])

# Remove duplicates
df = df.drop_duplicates()

# Remove rows with NaN values
df = df.dropna()

## Feature Engineering

In [7]:
# Add date
df["date"] = df["date_time"].dt.date

# Add the month
df["month"] = df["date_time"].dt.month

# Add the time of day
df["hour"] = df["date_time"].dt.hour

# Add the day of the week
df["day_of_week"] = df["date_time"].dt.dayofweek

# Add weekend or not
df['is_weekend'] = df['day_of_week'] >= 5

# Add the season based on the month (Germany)
df["season"] = df["month"].map({
    1: "winter",
    2: "winter",
    3: "spring",
    4: "spring",
    5: "spring",
    6: "summer",
    7: "summer",
    8: "summer",
    9: "autumn",
    10: "autumn",
    11: "autumn",
    12: "winter"
})

## Weather Data
https://open-meteo.com/en/docs/historical-weather-api#timezone=Europe%2FBerlin

In [13]:
# Coordinates of Karlsruhe
latitude = 49.014920
longitude = 8.390050

df_weather = utils.weather.get_weather_with_api(latitude=latitude, longitude=longitude, start_date=df["date_time"].min().strftime("%Y-%m-%d"), end_date=df["date_time"].max().strftime("%Y-%m-%d"))

In [14]:
df_weather

,date,hour,outside_tmp,outside_hum,outside_rain,outside_snowfall,outside_wind_speed,outside_pressure
0,2022-03-31,0,8.439,81.694,0.0,0.0,10.740,1000.5
1,2022-03-31,1,7.689,84.486,0.0,0.0,10.446,1000.0
2,2022-03-31,2,8.439,86.340,0.0,0.0,7.422,999.6
3,2022-03-31,3,8.089,87.806,0.0,0.0,8.855,999.1
4,2022-03-31,4,7.789,88.083,0.0,0.0,8.789,998.5
...,...,...,...,...,...,...,...,...
13075,2023-09-26,19,21.989,53.410,0.0,0.0,2.546,1020.6
13076,2023-09-26,20,20.389,59.308,0.0,0.0,2.742,1020.9
13077,2023-09-26,21,19.039,64.702,0.0,0.0,2.415,1021.4
13078,2023-09-26,22,15.689,78.647,0.0,0.0,5.352,1021.6


In [15]:
df_merged = pd.merge(df, df_weather, on=["date", "hour"], how="left")